<a href="https://colab.research.google.com/github/KeshekaE/EEG_Prediction_System/blob/main/EEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import pandas as pd
from torch import nn, optim
from tqdm import tqdm, trange
from torch.nn import functional as F
from sklearn.model_selection import train_test_split

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.l1 = nn.Linear(9, 128)
    self.l2 = nn.Linear(128, 1)
  
  def forward(self, x):
    x = F.relu(self.l1(x))
    return torch.sigmoid(self.l2(x))

In [ ]:
data = pd.read_csv('/EEGdata.csv', header=None)
x_train, x_test, y_train, y_test = train_test_split(
    data.iloc[:, :-1].to_numpy(),
    data.iloc[:, -1].map({'normal': 0, 'abnormal': 1}).to_numpy(),
    shuffle=True
  )

net = Net()
criterion = nn.MSELoss()
opt = optim.Adam(net.parameters(), lr=0.0001)

In [ ]:
for epoch in range(1):
  train_bar = tqdm(range(len(x_train)), position=0)
  for _ in train_bar:
    idxs = np.random.choice(range(len(x_train)), 64)
    x, y = torch.tensor(x_train[idxs], dtype=torch.float32), torch.tensor(y_train[idxs])


    y_hat = net(x)
    loss = criterion(y_hat.squeeze(dim=1), y.float())

    train_bar.set_description(f"loss = {loss.detach():0.4f}")
    opt.zero_grad()
    loss.backward()
    opt.step()

In [ ]:
# idx = 129
idx = 524

print(y_test[idx])
if y_test[idx] == 1:
    print("abnormal")
else:
    print("normal")
torch.bincount(net(torch.tensor(x_test, dtype=torch.float32)).long().squeeze(dim=1))

1
abnormal


tensor([    0, 10000])